<font size=6> Introduction to Machine Learning Homework </font>

<font size=6> 　　Lab5 - Recurrent Neural Network </font>

<p>本次作業主要分成兩個部份，第一部份要請同學們手刻一遍LSTM，加強理解其運作的核心想法；</p>
<p>第二部份則帶往實際應用面，練習去跑開源在github上的code，並有效率的修改替換所使用data。相關內容可以回顧 (4/28) 和 (5/01) 的課程影片。</p>

<font color=#0000ff size=5> 1. LSTM Implementation (50%)</font>

<p>在這邊我們的目標是透過numpy來實現LSTM(Long Short-Term Memory)網路的前向傳遞，請參考以下LSTM reference card</p>
<p>來完成input gate, output gate, forget gate這三個控制單元，並填入挖空的jupyter-notebook樣板中執行測資確認。</p>

![Image1](https://github.com/SundayDonghuLight/108-2_Introduction-to-Machine-Learning/blob/master/Lab5/figure/LSTM-ref-card.png?raw=true)

![Image2](https://github.com/SundayDonghuLight/108-2_Introduction-to-Machine-Learning/blob/master/Lab5/figure/LSTM-demo.png?raw=true)

In [1]:
import numpy as np 
from scipy.special import expit as sigmoid

<p>以下LSTM_demo class是不需要改動的部份，將被後續的LSTM class繼承。</p>
<p>這邊已經先設定好了網路的weights，請配合reference card和下方名稱用在各自對應的地方，</p>

In [2]:
class LSTM_demo():
    # Input size: 9  Hidden size: 2  Num layers: 1
    def __init__(self):
        # Forget gate weights
        self.W_hf = np.array([[0, 0], [0, 0]])
        self.B_hf = np.array([50, 50])
        self.W_xf = np.array([[0, 0, 0, 0, 0, 0, 0, -1000, -1000], [0, 0, 0, 0, 0, 0, 0, -1000, 0]])
        self.B_xf = np.array([50, 50])
        # Input gate weights
        self.W_hi = np.array([[0, 0], [0, 0]])
        self.B_hi = np.array([0, 0])
        self.W_xi = np.array([[50, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 50]])
        self.B_xi = np.array([0, 0])
        self.W_hl = np.array([[0, 0], [0, 0]])
        self.B_hl = np.array([0, 0])
        self.W_xl = np.array([[50, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 50]])
        self.B_xl = np.array([0, 0])
        # Output gate weights
        self.W_ho = np.array([[0, -500], [0, 0]])
        self.B_ho = np.array([0, 0])
        self.W_xo = np.array([[-1.33, 50, -0.78, -0.34, 0.5, 1.33, 0.01, 0, 0], [0, 0.65, -0.78, -0.34, 0.5, 0.08, 0, 0, -1.33]])
        self.B_xo = np.array([0, 0])
    
    def forget_gate(self, x, h, Weights_hf, Bias_hf, Weights_xf, Bias_xf, prev_cell_state): pass
    
    def input_gate(self, x, h, Weights_hi, Bias_hi, Weights_xi, Bias_xi, Weights_hl, Bias_hl, Weights_xl, Bias_xl): pass
    
    def cell_state(self, forget_gate_output, input_gate_output): pass
    
    def output_gate(self, x, h, Weights_ho, Bias_ho, Weights_xo, Bias_xo, cell_state): pass
    
    def classifier(self, h):
        if np.around(h[0], 3) == 0.208:
            return [0, 0, 0, 0, 0, 0, 1, 0, 0]
        elif np.around(h[1], 3) == 0.208:
            return [0, 0, 0, 0, 0, 0, 0, 1, 0]
        elif np.around(h[0]+h[1], 2) == 0.76:
            return [1, 0, 0, 0, 0, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.16:
            return [0, 1, 0, 0, 0, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.20:
            return [0, 0, 1, 0, 0, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.30:
            return [0, 0, 0, 1, 0, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.40:
            return [0, 0, 0, 0, 1, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.60:
            return [0, 0, 0, 0, 0, 1, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.50:
            return [0, 0, 0, 0, 0, 0, 0, 0, 1]
        
        elif np.around(h[0]+h[1], 2) == 0.23:
            return [0, 0, 0, 1, 1, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.41:
            return [0, 0, 0, 0, 1, 0, 1, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.48:
            return [0, 0, 1, 1, 0, 0, 1, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.19:
            return [0, 0, 0, 0, 0, 0, 1, 1, 0]
        
        else:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0]
        
    def __call__(self, x, h_c):
        h = h_c[0]
        c = h_c[1]
        
        f = self.forget_gate(x, h, self.W_hf, self.B_hf, self.W_xf, self.B_xf, c)
        i = self.input_gate(x, h, self.W_hi, self.B_hi, self.W_xi, self.B_xi, \
                             self.W_hl, self.B_hl, self.W_xl, self.B_xl,)
        c = self.cell_state(f, i)
        h = self.output_gate(x, h, self.W_ho, self.B_ho, self.W_xo, self.B_xo, c)
        
        h_c = (h, c)
        output = self.classifier(h)
        
        return output, h_c

In [3]:
class LSTM(LSTM_demo):
    def __init__(self):
        super().__init__()
        
    def forget_gate(self, x, h, Weights_hf, Bias_hf, Weights_xf, Bias_xf, prev_cell_state):
        # need to be implemented
        h_result = np.dot(Weights_hf, h) + Bias_hf
        x_result = np.dot(Weights_xf, x) + Bias_xf
        return np.multiply(prev_cell_state, sigmoid(h_result + x_result))
        
        
    
    def input_gate(self, x, h, Weights_hi, Bias_hi, Weights_xi, Bias_xi, Weights_hl, Bias_hl, Weights_xl, Bias_xl):
        # need to be implemented
        sig_result = sigmoid((np.dot(Weights_hi, h) + Bias_hi) + (np.dot(Weights_xi, x) + Bias_xi))
        tan_result = np.tanh((np.dot(Weights_hl, h) + Bias_hl) + (np.dot(Weights_xl, x) + Bias_xl))
        return np.multiply(sig_result, tan_result)
        
    
    def cell_state(self, forget_gate_output, input_gate_output):
        # need to be implemented
        new_cell_state = forget_gate_output + input_gate_output
        return new_cell_state
        
    
    def output_gate(self, x, h, Weights_ho, Bias_ho, Weights_xo, Bias_xo, cell_state):
        # need to be implemented
        tan_cell = np.tanh(cell_state)
        sig_out = sigmoid((np.dot(Weights_ho, h) + Bias_ho) + (np.dot(Weights_xo, x) + Bias_xo))
        output_h = np.multiply(tan_cell, sig_out)
        return output_h
        

<font size=4>測資介紹與LSTM：</font>
<p>"老吾老以及人之老，幼吾幼以及人之幼。"</p>
<p>我們將使用建構出的LSTM來預測上面這段國學名句，Input為第一個字'老'，而model將會預測出下一個字應該是接什麼，再把下一個丟進去即可預測下下個，依此類推。</p>
<p>在自然語言處理領域常會採用word embedding或稱word vector的技術，即把每一個字都對應到一個向量來表示，一種簡單的方式是使用one-hot encoding，總共有幾個不同的字就採用多少維的向量，每個字對應到只有一個元素為1其他為0的向量。</p>
<p>這邊定義以下兩個轉換函數</p>

In [4]:
def word2vector(word):
    if word == '老':
        vector = [1, 0, 0, 0, 0, 0, 0, 0, 0]
    elif word == '吾':
        vector = [0, 1, 0, 0, 0, 0, 0, 0, 0]
    elif word == '以':
        vector = [0, 0, 1, 0, 0, 0, 0, 0, 0]
    elif word == '及':
        vector = [0, 0, 0, 1, 0, 0, 0, 0, 0]
    elif word == '人':
        vector = [0, 0, 0, 0, 1, 0, 0, 0, 0]
    elif word == '之':
        vector = [0, 0, 0, 0, 0, 1, 0, 0, 0]
    elif word == '，':
        vector = [0, 0, 0, 0, 0, 0, 1, 0, 0]
    elif word == '。':
        vector = [0, 0, 0, 0, 0, 0, 0, 1, 0]
    elif word == '幼':
        vector = [0, 0, 0, 0, 0, 0, 0, 0, 1]
    else:
        vector = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    return vector

In [5]:
def vector2word(vector):
    if vector == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
        word = '老'
    elif vector == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
        word = '吾'
    elif vector == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
        word = '以'
    elif vector == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
        word = '及'
    elif vector == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
        word = '人'
    elif vector == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
        word = '之'
    elif vector == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
        word = '，'
    elif vector == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
        word = '。'
    elif vector == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
        word = '幼'
    else:
        word = 'UNKNOWN WORD'
    return word

<p>初始化hidden state與cell state為0 並執行測資</p>

In [6]:
lstm = LSTM()
h_c = (np.zeros(2), np.zeros(2))

In [7]:
string = ""
x = '老'
string = string + x

while(1):
    print(string)
    x = word2vector(x)
    output, h_c = lstm(x, h_c)
    x = vector2word(output)
    if x == 'UNKNOWN WORD': break
    string = string + x

老
老吾
老吾老
老吾老以
老吾老以及
老吾老以及人
老吾老以及人之
老吾老以及人之老
老吾老以及人之老，
老吾老以及人之老，幼
老吾老以及人之老，幼吾
老吾老以及人之老，幼吾幼
老吾老以及人之老，幼吾幼以
老吾老以及人之老，幼吾幼以及
老吾老以及人之老，幼吾幼以及人
老吾老以及人之老，幼吾幼以及人之
老吾老以及人之老，幼吾幼以及人之幼
老吾老以及人之老，幼吾幼以及人之幼。


<font size=4>分析：</font>

![image3](https://github.com/SundayDonghuLight/108-2_Introduction-to-Machine-Learning/blob/master/Lab5/figure/transition-diagram.png?raw=true)

<p>可以看出相同的input有可能產生不同的output(e.g.,老,幼,吾)，所以用一般的NN是沒辦法處理的，可以使用能記下過去資訊的RNN系列方法來解決。其中LSTM有著負責長期記憶的long-term-memory(cell state)和短期記憶的short-term-memory(hidden state)，像 以->及->人->之 只需要看前一個input就能推出下一個是什麼，主要靠著短期記憶的部份來處理，而'老'和'幼'的下一個字會接什麼可能就需要參考較長時間的記憶資訊。</p>
<p>以下從剛剛的例子來想像下LSTM三個控制單元的功能與可能做到的一些事，通常對於傳遞下去的cell state改變的較慢，只是調整下加上一些數值，而hidden state在不同節點往往有很大的區別，這邊的cell state就是拿來存'老'和'幼'出現過了幾次(認為需要長期記憶的資訊)。所以在處理時序問題時，hidden state常常拿來用作抽取出來的特徵，後續再丟到fully connected之類的網路中進行分類或其他應用。</p>
<p>Input gate: 控制選擇性記憶，若當前的input是需要長期記憶的就讓它可以加進cell state中記下來，若比較不重要就少記點。</p>
<p>Output gate: 控制輸出到下一個時間節點的hidden state，根據input和前一hidden state決定要從長期記憶的cell state取哪幾項及取多少。</p>
<p>Forget gate: 選擇性的忘記cell state內容，如下面第9到第10行，當不再需要記憶'老'出現幾次時就將其遺忘，方便更好的控制模型輸出。</p>

In [8]:
print('   input      old_hidden        old_cell        output_hidden')

for i, word in enumerate(string):
    x = word2vector(word)
    old_hc = h_c
    output, h_c = lstm(x, h_c)
    print('{:<2} ({},{:<24},{}) -> {}'.format(i, word, str(old_hc[0]), str(old_hc[1]), h_c[0]))

input      old_hidden        old_cell        output_hidden
0  (老,[0. 0.]                 ,[0. 0.]) -> [0.15929455 0.        ]
1  (吾,[0.15929455 0.        ] ,[1. 0.]) -> [0.76159416 0.        ]
2  (老,[0.76159416 0.        ] ,[1. 0.]) -> [0.2016354 0.       ]
3  (以,[0.2016354 0.       ]   ,[2. 0.]) -> [0.30301304 0.        ]
4  (及,[0.30301304 0.        ] ,[2. 0.]) -> [0.4008518 0.       ]
5  (人,[0.4008518 0.       ]   ,[2. 0.]) -> [0.60006796 0.        ]
6  (之,[0.60006796 0.        ] ,[2. 0.]) -> [0.76239218 0.        ]
7  (老,[0.76239218 0.        ] ,[2. 0.]) -> [0.20812502 0.        ]
8  (，,[0.20812502 0.        ] ,[3. 0.]) -> [0.50001499 0.        ]
9  (幼,[0.50001499 0.        ] ,[3. 0.]) -> [0.         0.15929455]
10 (吾,[0.         0.15929455] ,[0. 1.]) -> [0.         0.50037533]
11 (幼,[0.         0.50037533] ,[0. 1.]) -> [0.        0.2016354]
12 (以,[0.        0.2016354]   ,[0. 2.]) -> [0.         0.30301304]
13 (及,[0.         0.30301304] ,[0. 2.]) -> [0.        0.4008518]
14 (人,[0.  

<p>為了避免造成誤導再次聲明下，這邊只是舉個例子來表達LSTM是有可能做到類似這樣的行為的，實際上機器並不會用人類的方式思考，想像成先準備好了一個具備這種能力的模型，再透過最佳化loss function的方式調出最好最合適的參數。</p>

<font color=#0000ff size=5> 2. Rnn Application - Sequence to Sequence Chatbot (50%)</font>

Chatbot source code: [https://github.com/ywk991112/pytorch-chatbot](https://github.com/ywk991112/pytorch-chatbot)<br>
Ptt Chinese corpus: [https://github.com/zake7749/Gossiping-Chinese-Corpus](https://github.com/zake7749/Gossiping-Chinese-Corpus)

<p>這題要做的事情是把上面連結的Chatbot source code跑起來，然後這個聊天機器人的專案原本是跑在一個英文語料的電影字幕dataset上，改成可以train在上面連結PTT中文語料的dataset上。這邊要求使用<code>Gossiping-QA-Dataset.txt</code>這份資料，並將檔案放在<code>data</code>資料夾下。</p>
<p>由於training時間偏長，這邊並不要求大家把它train完，只要執行<code>python main.py -tr data/Gossiping-QA-Dataset.txt -la 1 -hi 512 -lr 0.0001 -it 50000 -b 64 -p 500 -s 1000</code>能跑起來就算完成。</p>


<font size=4>環境安裝注意事項：</font>

<p>基本上一般在github上的專案都會列出他所使用到的套件版本或是提供其他的環境需求，套件的部份大概常見下面這兩種方式：</p>
<p>1. 直接列出所需要用到的套件與其版本，通常只會列最重要的那幾個，然後版本沒什麼相容性問題的套件不太會列出其版本(如下圖)。常常有可能並沒有把所有會用到的套件都列上去，這時可以直接執行程式等報錯 no module name ... 再一個個安裝。</p>

![image4](https://github.com/SundayDonghuLight/108-2_Introduction-to-Machine-Learning/blob/master/Lab5/figure/list-requirement.png?raw=true)

<p>2. 會附上由pip導出的requirement.txt或由anaconda導出的.yaml檔(如下圖)，檔案裡會詳細列出所有該環境中的套件與其使用的版本，可直接執行指令安裝。pip導出的requirement.txt會包含到蠻多跟其他機器環境本身相關的東西，所以通常都沒辦法順利裝好會報錯。一種作法是等有出錯時再把文檔中該套件的那行註解掉重新執行，並反覆直到不再報錯為止；若是anaconda導出的.yaml檔通常會比較順利，若也出現問題則用同樣的方式處理就好好。</p>


![image5](https://github.com/SundayDonghuLight/108-2_Introduction-to-Machine-Learning/blob/master/Lab5/figure/yaml-requirement.png?raw=true)

<p>然後在使用pytorch或tensorflow等深度學習套件時，還需要注意配合的cuda, cudnn版本才能順利的使用GPU加速</p>
<p>以這份專案來說雖然沒有列，但pytorch 0.4.x通常是搭cuda9，而目前較新的pytorch 1.x.x版則可搭cuda9或cuda10，雖然我們沒有權限可以去改學校電腦上安裝的cuda和cudnn，但是可以透過Anaconda在虛擬環境中去使用需要的版本。</p>

<p>可以透過以下指令創建一個叫"chatbot"的虛擬環境，使用的python版本為3.6，並進到環境裡面</p>
<p><code>conda create -n chatbot python=3.6</code></p>
<p><code>conda activate chatbot</code></p>

然後建議透過PyTorch官網上的指令來安裝，比較不容易發生相容性的問題
[https://pytorch.org/get-started/previous-versions/](https://pytorch.org/get-started/previous-versions/)

<p><code>conda install pytorch=0.4.1 cuda90 -c pytorch</code></p>
<p><code>conda install pytorch=0.4.1 cuda92 -c pytorch</code></p>
<p>這邊可以選擇這兩個其中之一來安裝，雖然requirement是列0.4.0，但大版本編號沒變的情況下通常不太會有問題，可能還修正了0.4.0的部份問題。而後面cuda90就是安裝cuda9.0，然後這邊要特別注意一下，Nvidia RTX20XX系列顯卡在使用cuda9.0是會出現一些bug的，目前試了RTX2070和RTX2080都有發生(如下圖所示)，所以需要安裝cuda9.2的版本，而414的電腦目前看起來應該都是GTX1080，兩種在使用上都沒問題。</p>

![image6](https://github.com/SundayDonghuLight/108-2_Introduction-to-Machine-Learning/blob/master/Lab5/figure/CuDNN%20error:%20CUDNN_STATUS_SUCCESS.png?raw=true)

<p>環境都建好後，就可以跑github上的指令執行看看啦，記得學校伺服器是有使用Slurm工作調度工具來管理資源的，要透過srun向其發出執行工作的請求</p> 
<p><code>srun --gres=gpu:1 -u --x11=first python main.py -tr data/movie_subtitles.txt -la 1 -hi 512 -lr 0.0001 -it 50000 -b 64 -p 500 -s 1000</code></p>

[更多指令可參考](https://slurm.schedmd.com/pdfs/summary.pdf)

<font color=#0000ff size=5>作業繳交格式與相關規定</font>

<p>第1題請使用 ex1-template.ipynb 的檔案完成，只需要寫第3個block。</p>
<p>第2題從github上clone整個專案下來，把Gossiping-QA-Dataset.txt放到data資料夾下，直接修改code並確定以下指令<code>python main.py -tr data/Gossiping-QA-Dataset.txt -la 1 -hi 512 -lr 0.0001 -it 50000 -b 64 -p 500 -s 1000</code>能正確的執行，正確執行包含意義上的正確，假裝跑起來但裡面都亂寫被我抓到你就糟糕了XD，需注意data儲存格式與中英文轉word vector的差異。提醒下save/model下面會有存train過的checkpoints，那些不要上傳，檔案蠻大的。</p>
<p>最後將ex1-template.ipynb與專案的pytorch-chatbot資料夾一起壓成一個zip壓縮檔，檔名請用 <學號>.zip，並上傳到E3繳交。</p>

<font size=5>彩蛋</font>

<p>https://drive.google.com/file/d/1e7YYbHgEE7k2d____158sx8RV525QCtS/view?usp=sharing</p>
<p>在這個網址中放了第2題的提示，但中間被神秘的暗黑力量挖掉了一小塊，或許可以透過第一題的的LSTM還原?</p>
<p>PS. 不過盡量先自己改，做完再來看提示才會是比較好的練習，這已經是精挑細選還算蠻好讀的code了，且有可能同學們還會改的比我這邊更好。</p>

In [9]:
def word2vector(word):
    if word == 'd':
        vector = [1, 0, 0, 0, 1, 0, 0, 0, 0]
    elif word == 'Z':
        vector = [0, 0, 0, 0, 0, 0, 1, 1, 0]
    elif word == 's':
        vector = [0, 0, 0, 1, 1, 0, 0, 0, 0]
    elif word == 'y':
        vector = [0, 0, 1, 1, 0, 0, 1, 0, 0]
    elif word == 'h':
        vector = [0, 0, 0, 0, 1, 0, 1, 0, 0]
    else:
        vector = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    return vector

In [10]:
def vector2word(vector):
    if vector == [1, 0, 0, 0, 1, 0, 0, 0, 0]:
        word = 'd'
    elif vector == [0, 0, 0, 0, 0, 0, 1, 1, 0]:
        word = 'Z'
    elif vector == [0, 0, 0, 1, 1, 0, 0, 0, 0]:
        word = 's'
    elif vector == [0, 0, 1, 1, 0, 0, 1, 0, 0]:
        word = 'y'
    elif vector ==  [0, 0, 0, 0, 1, 0, 1, 0, 0]:
        word = 'h'
    else:
        word = 'UNKNOWN WORD'
    return word

In [11]:
lstm = LSTM()
h_c = (np.zeros(2), np.zeros(2))

In [18]:
string = ""
x = 'd' # <-- here

while(1):
    x = word2vector(x)
    output, h_c = lstm(x, h_c)
    x = vector2word(output)
    if x == 'UNKNOWN WORD': break
    string = string + x
    print(string)

s
sh
shy
shyZ


In [19]:
link = "https://drive.google.com/file/d/1e7YYbHgEE7k2d" + string + "158sx8RV525QCtS/view?usp=sharing"
print(link)

https://drive.google.com/file/d/1e7YYbHgEE7k2dshyZ158sx8RV525QCtS/view?usp=sharing
